# Load data

In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as K

In [2]:
def read_data(name):
    X_train, y_train = [], []
    X = np.load(f'data/keypoints/{name}.npy').reshape((-1, 51))
    with open(f'data/labels/{name}') as f:
        labels = f.readlines()

    N = int(re.findall(r'\d+', labels[0])[0])
    assert N == X.shape[0]

    end = 0

    for lab in labels:
        C = re.findall(r'\d:', lab)
        if len(C) == 1:
            C = int(C[0][0])

        idxs = re.findall(r'\d+-\d+', lab)
        for idx in idxs:
            start, stop = idx.split('-')
            start, stop = int(start), int(stop)

            X_train.append(X[end:start])
            y_train.append(0)

            X_train.append(X[start:stop])
            y_train.append(C)        

            end = stop

    X_train.append(X[end:-1])
    y_train.append(0)
    
    return X_train, y_train

In [3]:
X_train, y_train = read_data('id0_jab_1')

In [4]:
y_train = K.utils.to_categorical(y_train, num_classes=7)
X_train = tf.ragged.constant(X_train)

In [5]:
y_train.shape

(41, 7)

## Simple RNN 

In [6]:
inpu = K.layers.InputLayer((None, 51), batch_size=1)
rnn = K.layers.SimpleRNN(32, stateful=True)
dense1 = K.layers.Dense(64)
dense2 = K.layers.Dense(7, activation='softmax')

model = K.Sequential([inpu, rnn, dense1, dense2])

In [7]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (1, 32)                   2688      
_________________________________________________________________
dense (Dense)                (1, 64)                   2112      
_________________________________________________________________
dense_1 (Dense)              (1, 7)                    455       
Total params: 5,255
Trainable params: 5,255
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train, y_train, batch_size=1, epochs=50)

Epoch 1/50
41/41 [==============================] - 1s 4ms/step - loss: 1.3781 - accuracy: 0.4878
Epoch 2/50
41/41 [==============================] - 0s 4ms/step - loss: 1.1976 - accuracy: 0.3659
Epoch 3/50
41/41 [==============================] - 0s 4ms/step - loss: 1.1269 - accuracy: 0.4390
Epoch 4/50
41/41 [==============================] - 0s 4ms/step - loss: 1.1180 - accuracy: 0.5122
Epoch 5/50
41/41 [==============================] - 0s 4ms/step - loss: 1.0853 - accuracy: 0.5122
Epoch 6/50
41/41 [==============================] - 0s 4ms/step - loss: 1.1512 - accuracy: 0.4634
Epoch 7/50
41/41 [==============================] - 0s 4ms/step - loss: 1.0982 - accuracy: 0.4878
Epoch 8/50
41/41 [==============================] - 0s 4ms/step - loss: 1.0437 - accuracy: 0.5366
Epoch 9/50
41/41 [==============================] - 0s 4ms/step - loss: 1.0756 - accuracy: 0.4390
Epoch 10/50
41/41 [==============================] - 0s 4ms/step - loss: 1.0365 - accuracy: 0.4634
Epoch 11/50
41/41 [